In [2]:
import os
import numpy as np
import cv2
from matplotlib import pyplot as plt

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.models import resnet18, resnet34, resnet50

In [3]:
resnet = resnet18
model_path = 'data/classification-best/top_nobg_linspace_mse-best.pth'

device = "cuda:0"

model = resnet(pretrained=False)
fc_in_features = model.fc.in_features

model.fc = nn.Sequential(
    nn.Linear(fc_in_features, 1),
    #nn.Sigmoid()
)
model.load_state_dict(torch.load(model_path))
model.to(device)
model.eval()

preprocess = transforms.Compose([
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225]),
])

/home/gun/anaconda3/envs/diffuser/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/gun/anaconda3/envs/diffuser/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [4]:
def get_scores(inputs):
    x_numpy = np.concatenate(inputs).reshape(-1, 360, 480, 3)
    x_tp = x_numpy.transpose([0, 3, 1, 2])
    x_eval = torch.from_numpy(x_tp).type(torch.float)
    X_eval = preprocess(x_eval).to(device)
    scores = model(X_eval)[:, 0].detach().cpu().numpy()
    return scores

In [11]:
lognames = [lg for lg in os.listdir('data/logs') if lg!='.DS_Store']

In [12]:
score_per_user = []
for logname in lognames:
    log_dir = os.path.join('data/logs', logname)
    scenes = [s for s in os.listdir(log_dir) if 'nobg' in s and s.endswith('.png')]
    inputs = []
    for scene in scenes:
        scene_path = os.path.join(log_dir, scene)
        image = cv2.imread(scene_path)[:,:,::-1]/255
        inputs.append(image)
        
    scores = get_scores(inputs)
    score_per_user.append([logname, np.mean(scores)])

In [13]:
sorted(score_per_user, key=lambda k: k[1], reverse=True)

[['hogun', 0.81339765],
 ['wooseok', 0.80153465],
 ['junseo', 0.78254473],
 ['MineuiHong', 0.78091514],
 ['minjae', 0.76403475],
 ['DohyeongKim', 0.74855816],
 ['GunminLee', 0.71443623],
 ['hojun', 0.68300456],
 ['jeongho', 0.6560363],
 ['jaeyeon', 0.59356]]